# Creating a Cleaned Dataset for Snow Spotter (Generic)

#### Summary

Zooniverse data exports provide a massive csv file with a huge range of data. However, for data analysis purposes, we are only interested in a a few fields. This script describes the general approach to create a cleaned netCDF or csv file in python. The final cleaned file will have columns for datetime, median value, mean value, and mean threshold value. Note that this code is not specific to any water year, and is only a template upon which changes must be made for a unique project. For specific code, the Snow Spotter Data Cleaning repository contains jupyter notebook files for water years 2016-2022.

#### Import the following packages

In [19]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from datetime import datetime
import xarray as xr
import numpy as np
from dateutil.relativedelta import relativedelta

#### Import the data export and create a dataframe with only the subject_data and annotations columns. 

In [ ]:
df = pd.read_csv("filepath\\filename.csv", usecols = ['subject_data', 'annotations'])

#### The following code extracts the datetime from the subject_data column.

 * First, the cell is transformed into a dict so that we can call the 'metadata' key. 
 
 * If you label your metadata column appropriately, this will already be in the form Y_m_d_HMS. If you labeled your column in another way, you will have to adjust this value so that it only contains the date. 
 
 * We can then use strip the datetime, finishing creating a column with just datetimes which we will need for our analysis later on.
 
**Important Note:** When uploading images to Zooniverse, the choice of medatata label will have an impact on the following code. It is best practice to label the metadata with only the image date and time, as this will limit the number of steps needed to extract a datetime from the classifications export. However, if something else is chosen, additional lines of code can extract the date from the metadata. While not preferable, as long as the date is somewhere within the metadata label, it can be extracted. For instance, if I had labeled my metadata with with both location and date, such as "niwot3_2021_12_01_095605", I would add this line of code as shown below.  

```python
    metadata = subject_dict_simple['metadata']
    date = metadata[7:len(metadata)]
```
Choice of metadata label can also affect datetime extraction, and the strip datetime code can be adjusted to fit any datetime format. The code below assumes the metadata is in the form Year_Month_Day_HourMinuteSecond (such as 2021_12_01_095605).

In [18]:
arr = []
row_count = len(df.index)
def extract_meta_data(index):
    subject_data = df.iloc[index,1]
    #string to dict
    subject_dict = json.loads(subject_data)
    #shed outer layer of dict
    shedded_subject_dict = dict(ele for sub in subject_dict.values() for ele in sub.items())
    #extract the metadata, see important note in markdown above
    metadata = shedded_subject_dict_simple['metadata']
    #extract datetime, see important note in markdown above
    date_time = datetime.strptime(date, '%Y_%m_%d_%H%M%S')
    return date_time

#### The next function extracts the participant response

 * Once again, the cell must first be transformed into a dict so that we can call the 'value' key.
 
 * This value will be either "Yes", "No", or "Unsure." We need to convert this into a numerical value so that we can average and graph responses later on. We use a simple loop to convert responses of "Yes" to 1.0, "No" to 0.0, and "Unsure" or "It's dark" to NaN.

In [4]:
def extract_value(index):
    annotation = df.iloc[index,0]
    #convert to string
    str_annotation = str(annotation)
    #get rid of brackets around dict
    annotation_dict_str = str_annotation[1:len(str_annotation)-1]
    #string to dict
    annotation_dict = json.loads(annotation_dict_str)
    #grab yes or no value from dict 
    annotation_value = annotation_dict['value']

    if(annotation_value == "Yes"): 
        return 1
    elif(annotation_value == "No"): 
        return 0
    else:
        return None

#### Combine the extracted datetime and reponse into a new dataframe, then calculate and combine the median and mean response for each datetime. 

The resulting dataframe will have one column for datetime, one column for median_value, and one column for mean_value. In additiona

In [6]:
i = 0       
while (i < 65449):
    i = i + 1
    data = []
    data.append(extract_meta_data(i))
    data.append(extract_value(i))
    arr.append(data)
    
    
final = pd.DataFrame(arr, columns = ['datetime','value'])

#average 15 participant responses
median_final = final.groupby('datetime').median().reset_index()
mean_final = final.groupby('datetime').mean().reset_index()
combined_final = median_final
combined_final['mean_value'] = mean_final.value
combined_final.columns = ['datetime', 'median_value', 'mean_value']
combined_final['mean_threshold'] = np.where(combined_final['mean_value'] >= 0.9, 1, np.where(np.isnan(combined_final['mean_value']), np.nan, 0))
combined_final.head()

##### Save the cleaned data set as a netCDF file...

In [10]:
cleaned_export = combined_final.to_xarray()
cleaned_export.to_netcdf("filepath\\filename.nc")

##### ... or as a csv file.

In [ ]:
combined_final.to_csv("filepath\\filename.csv")  